In [1]:
import pandas as pd
import numpy as np
import os
import glob
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
from sklearn.model_selection import train_test_split,GridSearchCV
import pickle
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.metrics import accuracy_score, roc_curve, precision_score, recall_score, confusion_matrix, roc_auc_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from google.cloud import storage
import json
# from sklearn.ensemble import CatBoostClassifier

In [2]:
client = bigquery.Client()

print("Client creating using default project: {}".format(client.project))
query = """
 select * from `jm-ebg.EDA_MODEL.label_data_jan_to_march_0905` where event_date not like '202303%'
 """
query_job = client.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    # location="US",
)  # API request - starts the query
df = query_job.to_dataframe()
df.shape

Client creating using default project: jm-ebg


(88238, 23)

In [3]:

client = bigquery.Client()

print("Client creating using default project: {}".format(client.project))
query = """
 select * from `jm-ebg.EDA_MODEL.label_data_jan_to_march_0905` where event_date  like '202303%' 
 """
query_job = client.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    # location="US",
)  # API request - starts the query
df1 = query_job.to_dataframe()
df1.shape

Client creating using default project: jm-ebg


(14333, 23)

In [4]:
df['label'].value_counts()

0    78331
1     9907
Name: label, dtype: Int64

In [5]:
df.rename(columns={'timeOnSite_past': 'timeOnSite', 'day_of_week_past': 'day_of_week','device_category_past': 'device_category','mobile_brand_name_past': 'mobile_brand_name','operating_system_past': 'operating_system','browser_past': 'browser','region_past': 'region','city_past': 'city','ft_campaign_past': 'ft_campaign','ft_medium_past': 'ft_medium','ft_source_past': 'ft_source','utm_term_placement_past': 'utm_term_placement','os_version_past':'os_version'}, inplace=True)

In [6]:
df1.rename(columns={'timeOnSite_past': 'timeOnSite', 'day_of_week_past': 'day_of_week','device_category_past': 'device_category','mobile_brand_name_past': 'mobile_brand_name','operating_system_past': 'operating_system','browser_past': 'browser','region_past': 'region','city_past': 'city','ft_campaign_past': 'ft_campaign','ft_medium_past': 'ft_medium','ft_source_past': 'ft_source','utm_term_placement_past': 'utm_term_placement','os_version_past':'os_version'}, inplace=True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88238 entries, 0 to 88237
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ProspectId           88238 non-null  object 
 1   user_pseudo_id       88238 non-null  object 
 2   visitStartTime       88238 non-null  Int64  
 3   event_date           88238 non-null  object 
 4   timeOnSite           88238 non-null  float64
 5   day_of_week          88238 non-null  Int64  
 6   device_category      88238 non-null  object 
 7   mobile_brand_name    87850 non-null  object 
 8   operating_system     88234 non-null  object 
 9   os_version           88238 non-null  object 
 10  browser              88234 non-null  object 
 11  region               88238 non-null  object 
 12  city                 88238 non-null  object 
 13  ft_campaign          88185 non-null  object 
 14  ft_medium            88224 non-null  object 
 15  ft_source            88238 non-null 

In [8]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14333 entries, 0 to 14332
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ProspectId           14333 non-null  object 
 1   user_pseudo_id       14333 non-null  object 
 2   visitStartTime       14333 non-null  Int64  
 3   event_date           14333 non-null  object 
 4   timeOnSite           14333 non-null  float64
 5   day_of_week          14333 non-null  Int64  
 6   device_category      14333 non-null  object 
 7   mobile_brand_name    14269 non-null  object 
 8   operating_system     14333 non-null  object 
 9   os_version           14333 non-null  object 
 10  browser              14333 non-null  object 
 11  region               14332 non-null  object 
 12  city                 14326 non-null  object 
 13  ft_campaign          13986 non-null  object 
 14  ft_medium            13988 non-null  object 
 15  ft_source            13996 non-null 

In [9]:
int_cols = list(df.select_dtypes(include=[int]).columns)
int_cols.remove('label')

In [10]:
cat_cols = list(df.select_dtypes(include=['O']).columns)
float_cols = list(df.select_dtypes(include=[float]).columns)

In [11]:
data={}
unwanted = {'(none)','(direct)','(not set)','(Other)'}  

In [12]:
def analysis_count(column_name,number):
    return df.groupby([column_name])[column_name].size().reset_index(name='count') \
                     .sort_values(['count'], ascending=False) \
                     .head(number).reset_index(drop=True)
def analysis_column(column_name,number):
    return df.groupby([column_name])[column_name].size().reset_index(name='count') \
                     .sort_values(['count'], ascending=False) \
                     .head(number)[column_name].tolist()

In [13]:
##data -> dictionary 
data['device_category'] =  analysis_column('device_category',5)
data['device_category'] = [i for i in data['device_category']  if i not in unwanted] 
data['mobile_brand_name'] =  analysis_column('mobile_brand_name',10)
data['mobile_brand_name'] = [i for i in data['mobile_brand_name']  if i not in unwanted] 
data['operating_system'] =  analysis_column('operating_system',5)
data['operating_system'] = [i for i in data['operating_system']  if i not in unwanted] 
data['os_version'] =  analysis_column('os_version',10)
data['os_version'] = [i for i in data['os_version']  if i not in unwanted] 
data['browser'] =  analysis_column('browser',8)
data['browser'] = [i for i in data['browser']  if i not in unwanted]
data['city'] =  analysis_column('city',160)
data['city'] = [i for i in data['city']  if i not in unwanted]
data['region'] =  analysis_column('region',50)
data['region'] = [i for i in data['region']  if i not in unwanted] 
data['ft_campaign'] =  analysis_column('ft_campaign',70)
data['ft_campaign'] = [i for i in data['ft_campaign']  if i not in unwanted] 
data['ft_medium'] =  analysis_column('ft_medium',5)
data['ft_medium'] = [i for i in data['ft_medium']  if i not in unwanted] 
data['ft_source'] =  analysis_column('ft_source',10)
data['ft_source'] = [i for i in data['ft_source']  if i not in unwanted] 
data['utm_term_placement'] =  analysis_column('utm_term_placement',200)
data['utm_term_placement'] = [i for i in data['utm_term_placement']  if i not in unwanted]

# data['placement'] =  analysis_column("placement",150)
# data['placement'] = [i for i in data['placement']  if i not in unwanted]

# data['placement_past'] = [i for i in data['placement_past']  if i not in unwanted]

In [14]:
### preprocess data
df['device_category'].loc[df['device_category'].apply(lambda x: (x not in data['device_category']) )] = '(Others)'
df['mobile_brand_name'].loc[df['mobile_brand_name'].apply(lambda x: (x not in data['mobile_brand_name']) )] = '(Others)'
df['operating_system'].loc[df['operating_system'].apply(lambda x: (x not in data['operating_system']) )] = '(Others)'
df['os_version'].loc[df['os_version'].apply(lambda x: (x not in data['os_version']) )] = '(Others)'
df['browser'].loc[df['browser'].apply(lambda x: (x not in data['browser']) )] = '(Others)'
df['city'].loc[df['city'].apply(lambda x: (x not in data['city']) )] = '(Others)'
df['region'].loc[df['region'].apply(lambda x: (x not in data['region']) )] = '(Others)'
df['ft_campaign'].loc[df['ft_campaign'].apply(lambda x: (x not in data['ft_campaign']) )] = '(Others)'
df['ft_medium'].loc[df['ft_medium'].apply(lambda x: (x not in data['ft_medium']) )] = '(Others)'
df['ft_source'].loc[df['ft_source'].apply(lambda x: (x not in data['ft_source']) )] = '(Others)'
df['utm_term_placement'].loc[df['utm_term_placement'].apply(lambda x: (x not in data['utm_term_placement']) )] = '(Others)'

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [15]:
dict_filename = 'model/cat_dict_esd_RF.pkl'
pickle.dump(data, open(dict_filename, 'wb'))

In [16]:
cat_cols

['ProspectId',
 'user_pseudo_id',
 'event_date',
 'device_category',
 'mobile_brand_name',
 'operating_system',
 'os_version',
 'browser',
 'region',
 'city',
 'ft_campaign',
 'ft_medium',
 'ft_source',
 'utm_term_placement',
 'user_psudo_id_conv',
 'ProspectId_conv',
 'date_conv']

In [17]:
### replace the string int and float value
for i in cat_cols:
    df[i].fillna("Value_Not_Found", inplace = True)
for i in int_cols:
    df[i].fillna(-99999, inplace = True) 
for i in float_cols:
    df[i].fillna(-99999999.000, inplace = True) 

In [18]:
drop_cols = ['event_date','user_psudo_id_conv', 'ProspectId_conv', 'visitStartTime_conv',
       'date_conv', 'day_diff']
df = df.drop(drop_cols, axis=1)
df = df.drop_duplicates(keep='first')

In [19]:
df['label'].value_counts()

0    78331
1     9907
Name: label, dtype: Int64

In [20]:
cat_cols = list(df.select_dtypes(include=['O']).columns)
cat_cols

['ProspectId',
 'user_pseudo_id',
 'device_category',
 'mobile_brand_name',
 'operating_system',
 'os_version',
 'browser',
 'region',
 'city',
 'ft_campaign',
 'ft_medium',
 'ft_source',
 'utm_term_placement']

In [21]:
pip install imbalanced-learn

Note: you may need to restart the kernel to use updated packages.


In [22]:
import imblearn
print(imblearn.__version__)

0.10.1


In [23]:
#categorical value handling
le = LabelEncoder()
for  i in cat_cols:
    df[i] = le.fit_transform(df[i])
    pickle.dump(le, open("label_encode/{}-esd-RF.pkl".format(i), 'wb'))

In [24]:
X = df.drop(['label'], axis=1)
y = df.label

In [25]:
y = y.astype('float')

In [26]:
y=np.floor(pd.to_numeric(y, errors='coerce')).astype('Int64')
import numpy as np
y = y.fillna(0).astype(np.int64, errors='ignore')
X = X.fillna(0).astype(np.int64, errors='ignore')

In [27]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

In [28]:
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=.25, stratify=y)

In [29]:
from sklearn.preprocessing import StandardScaler

ss_train = StandardScaler()
X_train = ss_train.fit_transform(X_train)

ss_test = StandardScaler()
X_test = ss_test.fit_transform(X_test)

In [30]:
y=y.astype('int') 

In [31]:
#Fitting Logistic Regression to the training set  
from sklearn.linear_model import LogisticRegression  
classifier= LogisticRegression(random_state=0,max_iter=1000000)  
classifier.fit(X_train, y_train)  


LogisticRegression(max_iter=1000000, random_state=0)

In [32]:
y_pred= classifier.predict(X_test)

In [33]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_pred, y_test)

TN, FP, FN, TP = confusion_matrix(y_pred, y_test).ravel()

print('True Positive(TP)  = ', TP)
print('False Positive(FP) = ', FP)
print('True Negative(TN)  = ', TN)
print('False Negative(FN) = ', FN)

True Positive(TP)  =  16996
False Positive(FP) =  2587
True Negative(TN)  =  17601
False Negative(FN) =  1982


In [34]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_auc_score, f1_score



print ("----------------------")


print("accuracy : ", accuracy_score(y_pred, y_test))
print(confusion_matrix(y_pred, y_test))
print("roc auc : ", roc_auc_score(y_pred, y_test))
print("recall : ", recall_score(y_pred, y_test))
print("precision : ", precision_score(y_pred, y_test))

----------------------
accuracy :  0.8833426951948118
[[17601  2587]
 [ 1982 16996]]
roc auc :  0.8837089254359217
recall :  0.8955632838022974
precision :  0.8678956237552979


In [35]:
#Predicting the test set result  
y_pred= classifier.predict(X_test) 
y_pred1= classifier.predict_proba(X_test) 


In [36]:
y_pred=y_test

In [37]:
y_pred1=y_pred1[:,1]

In [38]:
y_pred=list(y_pred)

In [39]:
y_pred1
for i in range(y_pred1.size):
    if y_pred1[i]>0.755:
        y_pred1[i]=1
    elif y_pred1[i]<0.03:
        y_pred1[i]=0
    else:
        y_pred1[i]=-1

In [40]:
y_pred1

array([ 1.,  0., -1., ..., -1.,  1.,  1.])

In [41]:
a=[]
b=[]
for i in range(y_pred1.size):
    if y_pred1[i]==1:
        a.append(y_pred1[i])
        b.append(y_pred[i])
    elif y_pred1[i]==0:
        a.append(y_pred1[i])
        b.append(y_pred[i])
        

In [42]:
len(a)

25021

In [43]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(a, b)

TN, FP, FN, TP = confusion_matrix(a, b).ravel()

print('True Positive(TP)  = ', TP)
print('False Positive(FP) = ', FP)
print('True Negative(TN)  = ', TN)
print('False Negative(FN) = ', FN)

True Positive(TP)  =  15331
False Positive(FP) =  106
True Negative(TN)  =  8707
False Negative(FN) =  877


In [44]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_auc_score, f1_score



print ("----------------------")


print("accuracy : ", accuracy_score(a, b))
print(confusion_matrix(a, b))
print("roc auc : ", roc_auc_score(a, b))
print("recall : ", recall_score(a, b))
print("precision : ", precision_score(a, b))

----------------------
accuracy :  0.9607130010790935
[[ 8707   106]
 [  877 15331]]
roc auc :  0.9669316158463744
recall :  0.945890918065153
precision :  0.9931333808382458


In [45]:
X.shape

(156662, 16)

In [46]:
y_train.shape

(117496,)

In [47]:
y_test.shape

(39166,)

In [40]:
---------

SyntaxError: invalid syntax (567458762.py, line 1)

In [587]:
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification
from sklearn.naive_bayes import GaussianNB
from sklearn.calibration import CalibratedClassifierCV
logreg = LinearSVC(max_iter=1000000)
model = CalibratedClassifierCV(logreg) 
model.fit(X_train, y_train) 
predictions = model.predict(X_test)

ValueError: Unknown label type: 'unknown'

In [ ]:
y_proba = model.predict_proba(X_test)

In [ ]:
predictions=y_test

In [ ]:
y_proba=y_proba[:,1]

In [ ]:
predictions=list(predictions)

In [ ]:
for i in range(y_pred1.size):
    if y_proba[i]>0.755:
        y_proba[i]=1
    elif y_proba[i]<0.03:
        y_proba[i]=0
    else:
        y_proba[i]=-1

In [ ]:
y_proba

In [ ]:
c=[]
d=[]
for i in range(y_proba.size):
    if y_proba[i]==1:
        c.append(y_proba[i])
        d.append(predictions[i])
    elif y_proba[i]==0:
        c.append(y_proba[i])
        d.append(predictions[i])
        

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(c, d)

TN, FP, FN, TP = confusion_matrix(c, d).ravel()

print('True Positive(TP)  = ', TP)
print('False Positive(FP) = ', FP)
print('True Negative(TN)  = ', TN)
print('False Negative(FN) = ', FN)

In [ ]:
accuracy =  (TP + TN) / (TP + FP + TN + FN)

print('Accuracy of the binary classifier = {:0.3f}'.format(accuracy))

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_auc_score, f1_score



print ("----------------------")


print("accuracy : ", accuracy_score(c, d))
print(confusion_matrix(c, d))
print("roc auc : ", roc_auc_score(c, d))
print("recall : ", recall_score(c, d))
print("precision : ", precision_score(c, d))

In [52]:
pip install scikit-optimize

Note: you may need to restart the kernel to use updated packages.


In [53]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# Define the hyperparameter search space
search_space = {
    'C': Real(0.001, 100, prior='log-uniform'),
    'penalty': Categorical(['l1', 'l2']),
    'solver': Categorical(['liblinear', 'saga'])
}

# Define the logistic regression model
logreg = LogisticRegression(max_iter=100000)

# Define the Bayesian optimization search using 5-fold cross-validation
bayes_search = BayesSearchCV(
    logreg,
    search_space,
    n_iter=50,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=0,
    random_state=42
)

# Perform the hyperparameter search
bayes_search.fit(X_train, y_train)

# Evaluate the performance of the model on the test set
y_pred = bayes_search.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Test accuracy:', accuracy)

# Print the best hyperparameters found by Bayesian optimization
print('Best hyperparameters:', bayes_search.best_params_)

Test accuracy: 0.794889502762431
Best hyperparameters: OrderedDict([('C', 0.020172796349721606), ('penalty', 'l1'), ('solver', 'saga')])


In [ ]:
-----------

In [48]:
df1['label'].value_counts()

0    12673
1     1660
Name: label, dtype: Int64

In [49]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14333 entries, 0 to 14332
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ProspectId           14333 non-null  object 
 1   user_pseudo_id       14333 non-null  object 
 2   visitStartTime       14333 non-null  Int64  
 3   event_date           14333 non-null  object 
 4   timeOnSite           14333 non-null  float64
 5   day_of_week          14333 non-null  Int64  
 6   device_category      14333 non-null  object 
 7   mobile_brand_name    14269 non-null  object 
 8   operating_system     14333 non-null  object 
 9   os_version           14333 non-null  object 
 10  browser              14333 non-null  object 
 11  region               14332 non-null  object 
 12  city                 14326 non-null  object 
 13  ft_campaign          13986 non-null  object 
 14  ft_medium            13988 non-null  object 
 15  ft_source            13996 non-null 

In [50]:
int_cols = list(df1.select_dtypes(include=[int]).columns)
int_cols.remove('label')

In [51]:
cat_cols = list(df1.select_dtypes(include=['O']).columns)
float_cols = list(df1.select_dtypes(include=[float]).columns)

In [52]:
data={}
unwanted = {'(none)','(direct)','(not set)','(Other)'}  

In [53]:
def analysis_count(column_name,number):
    return df1.groupby([column_name])[column_name].size().reset_index(name='count') \
                     .sort_values(['count'], ascending=False) \
                     .head(number).reset_index(drop=True)
def analysis_column(column_name,number):
    return df1.groupby([column_name])[column_name].size().reset_index(name='count') \
                     .sort_values(['count'], ascending=False) \
                     .head(number)[column_name].tolist()

In [54]:
##data -> dictionary 
data['device_category'] =  analysis_column('device_category',5)
data['device_category'] = [i for i in data['device_category']  if i not in unwanted] 
data['mobile_brand_name'] =  analysis_column('mobile_brand_name',10)
data['mobile_brand_name'] = [i for i in data['mobile_brand_name']  if i not in unwanted] 
data['operating_system'] =  analysis_column('operating_system',5)
data['operating_system'] = [i for i in data['operating_system']  if i not in unwanted] 
data['os_version'] =  analysis_column('os_version',10)
data['os_version'] = [i for i in data['os_version']  if i not in unwanted] 
data['browser'] =  analysis_column('browser',8)
data['browser'] = [i for i in data['browser']  if i not in unwanted]
data['city'] =  analysis_column('city',160)
data['city'] = [i for i in data['city']  if i not in unwanted]
data['region'] =  analysis_column('region',50)
data['region'] = [i for i in data['region']  if i not in unwanted] 
data['ft_campaign'] =  analysis_column('ft_campaign',70)
data['ft_campaign'] = [i for i in data['ft_campaign']  if i not in unwanted] 
data['ft_medium'] =  analysis_column('ft_medium',5)
data['ft_medium'] = [i for i in data['ft_medium']  if i not in unwanted] 
data['ft_source'] =  analysis_column('ft_source',10)
data['ft_source'] = [i for i in data['ft_source']  if i not in unwanted] 
data['utm_term_placement'] =  analysis_column('utm_term_placement',200)
data['utm_term_placement'] = [i for i in data['utm_term_placement']  if i not in unwanted]

# data['placement'] =  analysis_column("placement",150)
# data['placement'] = [i for i in data['placement']  if i not in unwanted]

# data['placement_past'] = [i for i in data['placement_past']  if i not in unwanted]

In [55]:
### preprocess data
df['device_category'].loc[df['device_category'].apply(lambda x: (x not in data['device_category']) )] = '(Others)'
df['mobile_brand_name'].loc[df['mobile_brand_name'].apply(lambda x: (x not in data['mobile_brand_name']) )] = '(Others)'
df['operating_system'].loc[df['operating_system'].apply(lambda x: (x not in data['operating_system']) )] = '(Others)'
df['os_version'].loc[df['os_version'].apply(lambda x: (x not in data['os_version']) )] = '(Others)'
df['browser'].loc[df['browser'].apply(lambda x: (x not in data['browser']) )] = '(Others)'
df['city'].loc[df['city'].apply(lambda x: (x not in data['city']) )] = '(Others)'
df['region'].loc[df['region'].apply(lambda x: (x not in data['region']) )] = '(Others)'
df['ft_campaign'].loc[df['ft_campaign'].apply(lambda x: (x not in data['ft_campaign']) )] = '(Others)'
df['ft_medium'].loc[df['ft_medium'].apply(lambda x: (x not in data['ft_medium']) )] = '(Others)'
df['ft_source'].loc[df['ft_source'].apply(lambda x: (x not in data['ft_source']) )] = '(Others)'
df['utm_term_placement'].loc[df['utm_term_placement'].apply(lambda x: (x not in data['utm_term_placement']) )] = '(Others)'

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [56]:
### replace the string int and float value
for i in cat_cols:
    df1[i].fillna("Value_Not_Found", inplace = True)
for i in int_cols:
    df1[i].fillna(-99999, inplace = True) 
for i in float_cols:
    df1[i].fillna(-99999999.000, inplace = True) 

In [57]:
df.columns

Index(['ProspectId', 'user_pseudo_id', 'visitStartTime', 'timeOnSite',
       'day_of_week', 'device_category', 'mobile_brand_name',
       'operating_system', 'os_version', 'browser', 'region', 'city',
       'ft_campaign', 'ft_medium', 'ft_source', 'utm_term_placement', 'label'],
      dtype='object')

In [58]:
df1.columns

Index(['ProspectId', 'user_pseudo_id', 'visitStartTime', 'event_date',
       'timeOnSite', 'day_of_week', 'device_category', 'mobile_brand_name',
       'operating_system', 'os_version', 'browser', 'region', 'city',
       'ft_campaign', 'ft_medium', 'ft_source', 'utm_term_placement',
       'user_psudo_id_conv', 'ProspectId_conv', 'visitStartTime_conv',
       'date_conv', 'day_diff', 'label'],
      dtype='object')

In [59]:
drop_cols = ['event_date','user_psudo_id_conv', 'ProspectId_conv',
       'visitStartTime_conv', 'date_conv', 'day_diff']
df1 = df1.drop(drop_cols, axis=1)
df1= df1.drop_duplicates(keep='first')

In [60]:
df1['label'].value_counts()

0    12673
1     1660
Name: label, dtype: Int64

In [61]:
df1 = df1.reindex(columns=df.columns)

In [62]:
cat_cols = list(df1.select_dtypes(include=['O']).columns)
cat_cols

['ProspectId',
 'user_pseudo_id',
 'device_category',
 'mobile_brand_name',
 'operating_system',
 'os_version',
 'browser',
 'region',
 'city',
 'ft_campaign',
 'ft_medium',
 'ft_source',
 'utm_term_placement']

In [63]:
#categorical value handling
le = LabelEncoder()
for  i in cat_cols:
    df1[i] = le.fit_transform(df1[i])
    pickle.dump(le, open("label_encode/{}-esd-RF.pkl".format(i), 'wb'))

In [64]:
X = df1.drop(['label'], axis=1)
y = df1.label

In [65]:
from sklearn.preprocessing import StandardScaler

ss_train = StandardScaler()
X_train = ss_train.fit_transform(X_train)

ss_test = StandardScaler()
X_test = ss_test.fit_transform(X_test)

In [421]:
y=np.floor(pd.to_numeric(y, errors='coerce')).astype('Int64')
import numpy as np
y = y.fillna(0).astype(np.int64, errors='ignore')
X = X.fillna(0).astype(np.int64, errors='ignore')

In [320]:
seed = 101
min_class_df = df1[df1['label']==1]
max_class_df = df1[df1['label']==0]
n_samples = int(min_class_df.shape[0])
minority_df_sampled = resample(min_class_df, n_samples=n_samples, replace=False, random_state=seed )
majority_df_sampled = resample(max_class_df, n_samples=n_samples, replace=False, random_state=seed)
sampled_df = pd.concat([minority_df_sampled, majority_df_sampled]).sample(frac=1, random_state=seed)

In [321]:
X = sampled_df.drop(['label'], axis=1)
y = sampled_df.label

In [422]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

In [66]:
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=.25, stratify=y)

In [67]:
y_pre=classifier.predict(X_test)

/opt/conda/lib/python3.7/site-packages/sklearn/base.py:444: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [100]:
y_pred2=classifier.predict(X_train)

/opt/conda/lib/python3.7/site-packages/sklearn/base.py:444: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [69]:
y_pre

array([1, 1, 1, ..., 1, 1, 1])

In [70]:
y

0        0
1        0
2        0
3        0
4        0
        ..
14328    0
14329    0
14330    0
14331    0
14332    0
Name: label, Length: 14333, dtype: Int64

In [71]:
df1['label'].value_counts()

0    12673
1     1660
Name: label, dtype: Int64

In [72]:
df.head()

,ProspectId,user_pseudo_id,visitStartTime,timeOnSite,day_of_week,device_category,mobile_brand_name,operating_system,os_version,browser,region,city,ft_campaign,ft_medium,ft_source,utm_term_placement,label
0,68,14277,1675325695,0.682301,5,(Others),(Others),(Others),(Others),(Others),(Others),(Others),(Others),(Others),(Others),(Others),0
1,99,9211,1675582360,35.725837,1,(Others),(Others),(Others),(Others),(Others),(Others),(Others),(Others),(Others),(Others),(Others),0
2,132,10015,1675254861,0.462754,4,(Others),(Others),(Others),(Others),(Others),(Others),(Others),(Others),(Others),(Others),(Others),0
3,151,4589,1677211898,24.002012,6,(Others),(Others),(Others),(Others),(Others),(Others),(Others),(Others),(Others),(Others),(Others),0
4,222,3083,1676808404,0.868286,1,(Others),(Others),(Others),(Others),(Others),(Others),(Others),(Others),(Others),(Others),(Others),0


In [73]:
X.head()

,ProspectId,user_pseudo_id,visitStartTime,timeOnSite,day_of_week,device_category,mobile_brand_name,operating_system,os_version,browser,region,city,ft_campaign,ft_medium,ft_source,utm_term_placement
0,35,672,1678425248,4.844952,6,1,25,0,5,3,22,111,0,0,0,33
1,45,1682,1677610930,0.329827,4,1,25,0,6,2,19,16,26,2,9,33
2,120,709,1678636767,0.308013,1,1,25,0,3,3,35,67,69,2,10,171
3,228,2658,1678550505,0.026153,7,1,25,0,4,3,13,4,12,2,10,33
4,275,908,1678440013,1.799678,6,1,16,0,5,3,44,151,10,2,10,33


In [74]:
#Predicting the test set result  
y_pre= classifier.predict(X) 
y_pre1= classifier.predict_proba(X_test) 


/opt/conda/lib/python3.7/site-packages/sklearn/base.py:444: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:444: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [75]:
y_pre=y_test

In [76]:
y_pre1=y_pre1[:,1]

In [77]:
y_pre=list(y_pre)

In [78]:
y_pre1
for i in range(y_pre1.size):
    if y_pre1[i]>0.50:
        y_pre1[i]=1
    elif y_pre1[i]<0.50:
        y_pre1[i]=0
    else:
        y_pre1[i]=-1

In [79]:
y_pre1

array([1., 1., 1., ..., 1., 1., 1.])

In [80]:
a=[]
b=[]
for i in range(y_pre1.size):
    if y_pre1[i]==1:
        a.append(y_pre1[i])
        b.append(y_pre[i])
    elif y_pre1[i]==0:
        a.append(y_pre1[i])
        b.append(y_pre[i])

In [81]:
len(a)

3584

In [101]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_pre, y_test)

TN, FP, FN, TP = confusion_matrix(y_pre, y_test).ravel()

print('True Positive(TP)  = ', TP)
print('False Positive(FP) = ', FP)
print('True Negative(TN)  = ', TN)
print('False Negative(FN) = ', FN)

True Positive(TP)  =  415
False Positive(FP) =  0
True Negative(TN)  =  3169
False Negative(FN) =  0


In [98]:
y_test=y_test.astype('int64')
y_train=y_train.astype('int64')

In [102]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_auc_score, f1_score

print("accuracy : ", accuracy_score(y_pre, y_test))
print(confusion_matrix(y_pre, y_test))
print("roc auc : ", roc_auc_score(y_pre, y_test))
print("recall : ", recall_score(y_pre, y_test))
print("precision : ", precision_score(y_pre, y_test))

print ("----------------------")


print("accuracy : ", accuracy_score(y_pred2, y_train))
print(confusion_matrix(y_pred2, y_train))
print("roc auc : ", roc_auc_score(y_pred2, y_train))
print("recall : ", recall_score(y_pred2, y_train))
print("precision : ", precision_score(y_pred2, y_train))

accuracy :  1.0
[[3169    0]
 [   0  415]]
roc auc :  1.0
recall :  1.0
precision :  1.0
----------------------
accuracy :  0.11582472788166341
[[   0    0]
 [9504 1245]]


ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [84]:
y_test

11309    0
6810     0
3864     1
4708     0
9865     1
        ..
9004     0
4304     0
8397     0
6184     0
11581    0
Name: label, Length: 3584, dtype: Int64

In [85]:
y_pred2

array([1, 1, 1, ..., 1, 1, 1])

In [86]:
y_train

11242    0
9209     0
2608     0
953      0
11090    0
        ..
13833    0
6920     0
5512     1
9278     0
14100    0
Name: label, Length: 10749, dtype: Int64

In [87]:
df1.columns

Index(['ProspectId', 'user_pseudo_id', 'visitStartTime', 'timeOnSite',
       'day_of_week', 'device_category', 'mobile_brand_name',
       'operating_system', 'os_version', 'browser', 'region', 'city',
       'ft_campaign', 'ft_medium', 'ft_source', 'utm_term_placement', 'label'],
      dtype='object')